#[PyTorch Tutorial](https://pytorch.org/tutorials/index.html)

In [ ]:
%matplotlib inline


Training a Classifier
=====================

We will do the following steps in order:

1. Load and normalizing the MNIST training and test datasets using
   ``torchvision``
2. Define a Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

Using ``torchvision``, it’s extremely easy to load MNIST.



## MNIST dataset
- databaset of handwritten digits
- a training set of 60,000 examples
- a test set of 10,000 examples

![MNIST](https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn

## 1. Load and normalizing the datasets using torchvision

The output of torchvision datasets are "PIL Image" images of range [0, 1].

We transform them to Tensors of normalized range [-1, 1].



In [ ]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])

trainset = torchvision.datasets.MNIST(root='./data', 
                                      train=True,
                                      download=True, 
                                      transform=transform)
trainloader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=64,
                                          shuffle=True)

testset = torchvision.datasets.MNIST(root='./data',
                                     train=False,
                                     download=True, 
                                     transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=64,
                                         shuffle=False)

Let us show some of the training images, for fun.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(npimg[0])


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(images[0])
print('GroundTruth :', labels[0].item())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid

# functions to show an image

def imshow(imgs):
    imgs = imgs / 2 + 0.5     # unnormalize
    imgs = make_grid(imgs, nrow=8, padding=2, normalize=True, pad_value=255)
    npimgs = imgs.numpy()
    npimgs = np.transpose(npimgs, (1, 2, 0))
    plt.imshow(npimgs)


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(images)

## 2. Define a Convolution Neural Network



![network](https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/300px-Colored_neural_network.svg.png)

net = nn.Sequential(
        nn.Linear(3, 4),
        nn.ReLU(),
        nn.Linear(4, 2)
)

In [ ]:
net = nn.Sequential(nn.Linear(28 * 28, 128),
                    nn.ReLU(),
                    nn.Linear(128, 10))

## 3. Define a Loss function and optimizer

Let's use a Classification Cross-Entropy loss and SGD with momentum.
- Explanation about [Cross-Entropy](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html)
- Explanation about [Stochastic Gradient Descent](https://ruder.io/optimizing-gradient-descent/index.html#stochasticgradientdescent)



In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=net.parameters(), lr=0.001, momentum=0.9)

## 4. Train the network

This is when things start to get interesting.
We simply have to loop over our data iterator, and feed the inputs to the
network and optimize.



In [ ]:
import time

epochs = 5

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    start = time.time()
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs.view(inputs.size(0), -1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    # print statistics
    end = time.time()
    print('[%d] loss: %.3f time: %.3f' %
          (epoch + 1, running_loss / len(trainloader), (end-start)))
    start = time.time()

print('Finished Training')

## 5. Test the network on the test data

We have trained the network for 5 passes over the training dataset.
But we need to check if the network has learnt anything at all.

We will check this by predicting the class label that the neural network
outputs, and checking it against the ground-truth. If the prediction is
correct, we add the sample to the list of correct predictions.

Okay, first step. Let us display an image from the test set to get familiar.



In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(images[0])

outputs = net(images.view(images.size(0), -1))
_, predicted = torch.max(outputs.data, 1)

print('GroundTruth :', labels[0].item())
print('Predicted :', predicted[0].item())

Okay, now let us see what the neural network thinks these examples above are:



The results seem pretty good.

Let us look at how the network performs on the whole dataset.



In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images.view(images.size(0), -1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Okay, so what next?

How do we run these neural networks on the GPU?

Training on GPU
----------------
Just like how you transfer a Tensor on to the GPU, you transfer the neural
net onto the GPU.

Let's first define our device as the first visible cuda device if we have
CUDA available:



![CNN](https://miro.medium.com/max/1644/1*uAeANQIOQPqWZnnuH-VEyw.jpeg)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=4*4*50, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time

net = Net()
net = net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

epochs = 5

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    start = time.time()
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    # print statistics
    end = time.time()
    print('[%d] loss: %.3f time: %.3f' %
          (epoch + 1, running_loss / len(trainloader), (end-start)))
    start = time.time()

print('Finished Training')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))